In [1]:
import os
import numpy as np
import tifffile
import PIL
from PIL import Image, ImageSequence
from ScanImageTiffReader import ScanImageTiffReader, ScanImageTiffReaderContext
import imagej
import pandas as pd
import skimage

In [2]:
def get_file_name(path,kind):  
    os.chdir(path)
    files=[]
    for file in os.listdir():
        if file.endswith(kind):
            files.append(file)
    return files
def get_pos(image_file_name):
    start=image_file_name.find('_')+1
    end=image_file_name.find('_',4)
    return image_file_name[start:end]
def get_row(image_file_name):
    start=image_file_name.find('_',10)+1
    end=start+3
    return int(image_file_name[start:end])

def get_col(image_file_name):
    start=image_file_name.find('_',7)+1
    end=start+3
    return int(image_file_name[start:end])

In [5]:
#arrange folders
# pos_path=os.path.join("D:\\","20230322_Rabies_control_test_666346_1")
# image_folder='GFP01'
# channal_number=3
# file=get_file_name(os.path.join(pos_path,image_folder),'.tif')
# pos=[get_pos(f) for f in file]
# pos_ls=np.unique(pos)
# c_ls=["gfp","t","dic"]
# for i in c_ls:
#     if not os.path.exists(os.path.join(pos_path,i)):
#         os.mkdir(os.path.join(pos_path,i))        
# for i in c_ls:
#     for j in pos_ls:
#         if not os.path.exists(os.path.join(pos_path,i,j)):
#             os.mkdir(os.path.join(pos_path,i,j)) 
#         if not os.path.exists(os.path.join(pos_path,i,"stitched_"+j)):
#             os.mkdir(os.path.join(pos_path,i,"stitched_"+j))
# pos_index_ls=[]
# for pos_name in pos_ls:
#     index=[i for i, x in enumerate(pos) if x == pos_name]
#     pos_index_ls.append(index)
# for i in range(len(pos_index_ls)):
#     folder=pos_ls[i]
#     name=[file[index] for index in pos_index_ls[i]]
#     for n in name:
#         img=tifffile.imread(os.path.join(pos_path,image_folder,n))
#         for j in range(len(img)):
#             img_single=Image.fromarray(img[j])
#             img_single.save(os.path.join(pos_path,c_ls[j],folder,n))

In [3]:
ij=imagej.init('E:\\Allenwork\\Software\\Fiji.app')

In [4]:
ij.getVersion()

'2.9.0/1.53t'

In [ ]:
stitch_channal="dic"
image_file_name_ls=os.listdir(os.path.join(pos_path,stitch_channal,pos_ls[1]))
row=[get_row(i) for i in image_file_name_ls]
col=[get_col(i) for i in image_file_name_ls]
Height=max(row)+1
Width=max(col)+1
Starting_point="Upper Right"
img_directory=os.path.join(pos_path,stitch_channal,pos_ls[0])
img_save_directory=os.path.join(pos_path,stitch_channal,"stitched_"+pos_ls[1])
filenamePattern='Max_'+pos_ls[0]+'_\{ccc\}_\{rrr\}.tif'

In [ ]:
img_directory = img_directory.replace('\\','\\\\')
img_save_directory=img_save_directory.replace('\\','\\\\')

In [ ]:
macro = f'''run("MIST",\
"gridwidth={Width} \
gridheight={Height} \
starttilerow=0 \
starttilecol=0 \
imagedir={img_directory} \
filenamepattern={filenamePattern} \
filenamepatterntype=ROWCOL \
gridorigin=UR \
assemblefrommetadata=false \
assemblenooverlap=false \
globalpositionsfile=[] \
numberingpattern=HORIZONTALCOMBING \
startrow=0 \
startcol=0 \
extentwidth=8 \
extentheight=10 \
timeslices=0 \
istimeslicesenabled=false \
outputpath={img_save_directory} \
displaystitching=true outputfullimage=true outputmeta=true outputimgpyramid=false \
blendingmode=AVERAGE blendingalpha=NaN compressionmode=UNCOMPRESSED outfileprefix=img- \
unit=MICROMETER unitx=1.0 unity=1.0 programtype=JAVA numcputhreads=8 loadfftwplan=true \
savefftwplan=true fftwplantype=MEASURE fftwlibraryname=libfftw3 \
fftwlibraryfilename=libfftw3.dll planpath=C:\\\\Fiji.app\\\\lib\\\\fftw\\\\fftPlans \
fftwlibrarypath=C:\\\\Fiji.app\\\\lib\\\\fftw stagerepeatability=0 \
horizontaloverlap=10.0 \
verticaloverlap=10.0 \
numfftpeaks=0 \
overlapuncertainty=NaN \
isusedoubleprecision=false \
isusebioformats=false \
issuppressmodelwarningdialog=false \
isenablecudaexceptions=false \
translationrefinementmethod=SINGLE_HILL_CLIMB \
numtranslationrefinementstartpoints=16 \
headless=false \
loglevel=MANDATORY \
debuglevel=NONE");'''

In [ ]:
ij.py.run_macro(macro)

In [ ]:
global_positions_filepath=os.path.join('D:\\','20230322_Rabies_control_test_666346_1','dic','stitched_Pos1','img-global-positions-0.txt')
images_dirpath=os.path.join('D:\\','20230322_Rabies_control_test_666346_1','dic','Pos1')
stitch_images_dirpath=os.path.join('D:\\','20230322_Rabies_control_test_666346_1','t','Pos1')
stitch_images_savedirpath=os.path.join('D:\\','20230322_Rabies_control_test_666346_1','t','stitched_Pos1')

In [ ]:
img_names = list()
pixel_x_position = list()
pixel_y_position = list()
with open(global_positions_filepath, 'r') as fh:
    for line in fh:
        line = line.strip()
        toks = line.split(';')

        # name loading
        fn_tok = toks[0]
        fn = fn_tok.split(':')[1].strip()
        img_names.append(fn)

        # position loading
        pos_tok = toks[2]
        pos_pair = pos_tok.split(':')[1].strip()
        pos_pair = pos_pair.replace(')', '')
        pos_pair = pos_pair.replace('(', '')
        pos_pairs = pos_pair.split(',')
        x = int(pos_pairs[0].strip())
        y = int(pos_pairs[1].strip())
        pixel_x_position.append(x)
        pixel_y_position.append(y)


In [ ]:
d={'img_names':img_names,'pixel_x_position':pixel_x_position,'pixel_y_position':pixel_y_position}
trans = pd.DataFrame(data=d)
trans.to_csv(os.path.join('D:\\','20230322_Rabies_control_test_666346_1','dic','stitched_Pos1','Pos1'+'_transformation.csv'), index=False)

In [ ]:
first_tile = skimage.io.imread(os.path.join(images_dirpath, img_names[0]))
tile_shape = first_tile.shape
n_channels = 1
if len(tile_shape) == 3:
    n_channels = tile_shape[2]
tile_h = tile_shape[0]
tile_w = tile_shape[1]

stitched_img_h = tile_h + np.max(pixel_y_position)
stitched_img_w = tile_w + np.max(pixel_x_position)

In [ ]:
stitched_img = np.zeros((stitched_img_h, stitched_img_w), dtype=first_tile.dtype)

In [ ]:
for i in range(0, len(img_names)):
    fn = img_names[i]
    x = pixel_x_position[i]
    y = pixel_y_position[i]

    tile = skimage.io.imread(os.path.join(stitch_images_dirpath, fn))
    if tile.shape != tile_shape:
        raise RuntimeError('All images must be the same shape. Image {} is {}, expected {}'.format(fn, tile.shape, tile_shape))
    if tile.dtype != first_tile.dtype:
        raise RuntimeError('Img {} has type: {}, expected {}.'.format(fn, tile.dtype, first_tile.dtype))

    stitched_img[y:y+tile_h, x:x+tile_w] = tile
skimage.io.imsave(os.path.join(stitch_images_savedirpath,'stitched_'+'Pos1.tif'),stitched_img,photometric='minisblack')